In [1]:
from lxml import etree
from pathlib import Path
from pymongo import MongoClient

In [2]:
db=MongoClient()

In [3]:
stations=db.stations_non_germany_2020

In [4]:
stations.create_collection("logs")
logs_doc=stations.logs

In [5]:
sorted(list(stations.list_collections()),key=lambda station:station['name'])

[{'name': 'logs',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('cac0133d-b653-4ef1-9f7b-99af047b17e8')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [6]:
def get_safe_first(values):
    if values:
        return values[0]
    else:
        return ""

In [7]:
path = Path("./shows")

In [8]:
files=[file for file in path.iterdir() if not file.is_dir()]

In [9]:
for file in files:
    if file.stem not in stations.list_collection_names():
        stations.create_collection(file.stem)
        print(f"collection {file.stem} created")

collection 2020_3SAT created
collection 2020_ALPHA created
collection 2020_ARD created
collection 2020_ARTE created
collection 2020_BFS created
collection 2020_DMAX created
collection 2020_DW-tv created
collection 2020_EinsFestival created
collection 2020_HF created
collection 2020_K1 created
collection 2020_K1DOKU created
collection 2020_KKA created
collection 2020_MDR created
collection 2020_N24DOKU created
collection 2020_N3_Hamburg created
collection 2020_N3_Radio_Bremen created
collection 2020_NTV created
collection 2020_PHOENIX created
collection 2020_PRO7 created
collection 2020_ProSiebenFun created
collection 2020_ProSieben_MAXX created
collection 2020_rbb_Berlin created
collection 2020_RTL created
collection 2020_RTL2 created
collection 2020_RTLplus created
collection 2020_RTLS created
collection 2020_RTL_Nitro created
collection 2020_SAT1 created
collection 2020_Sat1Gold created
collection 2020_sixx created
collection 2020_Sport1 created
collection 2020_SWR_BW created
collect

In [10]:
parser = etree.XMLParser(recover=True)

In [11]:
def save_show(show,station,extras={}):
    params={
            'show_id':show.xpath("./sendung//stammdaten//metadaten/id/text()")[0],
            'title':show.xpath("./sendung//stammdaten//programm/titel/text()")[0],
    }
    values={**params,**extras}
    values['subtitle']=get_safe_first(show.xpath("./sendung//stammdaten//programm/nebentitel/text()")) 
    values['originaltitel']=get_safe_first(show.xpath("./sendung//stammdaten//programm/originaltitel/text()")) 
    values['sender_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/sender_genre/text()"))
    values['dmb_haupt_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/dmb_genre/dmb_haupt_genre/text()"))
    values['dmb_neben_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/dmb_genre/dmb_neben_genre/text()"))
    values['produktion_land']=get_safe_first(show.xpath("./sendung//stammdaten//programm/produktion/land/text()"))
    values['produktion_jahr']=get_safe_first(show.xpath("./sendung//stammdaten//programm/produktion/jahr/text()"))
    values['hinweissender']=get_safe_first(show.xpath("./sendung//stammdaten//programm/hinweissender/text()"))
    values['titel_ergaenzung']=get_safe_first(show.xpath("./sendung//stammdaten//programm/titel_ergaenzung/text()"))
    values['hinweissender']=get_safe_first(show.xpath("./sendung//stammdaten//programm/hinweissender/text()"))
    values['ausstrahlung_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/start/text()"))
    values['ausstrahlung_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/wh-text/text()"))
    values['ausstrahlung_sender_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/sender_start/text()"))
    values['ausstrahlung_stop']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/stop/text()"))
    values['ausstrahlung_dauer']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/dauer/text()"))
    values['nettolaenge']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/nettolaenge/text()"))
    values['ausstrahlung_sender']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/sender/text()"))
    values['produzierender_sender']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/produzierender_sender/text()"))
    values['dmbInfoset_ProduktionsLaender_ProduktionsLand']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/ProduktionsLaender/ProduktionsLand/text()"))
    values['dmbFolgetitel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbFolgetitel/text()"))
    values['dmbStaffel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbStaffel/text()"))
    values['dmbFolge']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbFolge/text()"))
    values['dmbProduktionsJahr']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbProduktionsJahr/text()"))
    values['dmbOriginalFolgetitel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbOriginalFolgetitel/text()"))
    values['staff']=show.xpath("./sendung//stammdaten//dmbInfoset/StaffList/Staff/Person/Fullname/text()")
    produktionsLand_nodes=show.xpath("./sendung//stammdaten//dmbInfoset/ProduktionsLaender/ProduktionsLand")
    produktionsLand=[{"land":node.text,'kfz':node.attrib.get("dmbstd:kfz"),'alpha3':node.attrib.get("dmbstd:alpha3") } for node in produktionsLand_nodes]
    values['produktionsLand']=produktionsLand
    staff_nodes=show.xpath("./sendung//stammdaten//dmbInfoset/StaffList/Staff")
    staff=[{"staff_id":node.xpath("./Person")[0].attrib.get("dmbstd:id"),"staff_function":node.attrib.get("dmbstd:Function"),"fullname":node.xpath("./Person/Fullname")[0].text} for node in staff_nodes]
    values['staff']=staff
    values['crew_united_url']=""
    values['crew_united_productions']=[]
    values['crew_united_koproduzent_productions']=[]
    values['cu_first_showing_country']=""
    values['cu_first_showing_station']=""
    values['cu_first_showing_date_time']=""
    values['imdb_id']=""
    values['imdb_link']=""
    values['imdb_distributors']=[]
    values['imdb_productions']=[]
    values['imdb_flag']=""
    values['cu_flag']=""
    values["fernsehserien_url"]=""
    values[ "fernsehserien_title"]=""
    values["fernsehserien_episode_title"]=""
    values["fernsehserien_serie_number"]=""
    values["fernsehserien_duration"]=""
    values["fernsehserien_production_companies"]=[]
    values["fernsehserien_released_at"]=[]
    
    
    
    station.replace_one(params,values,upsert=True)

In [12]:
for file in files:
    if file.stem not in stations.list_collection_names():
        stations.create_collection(file.stem)
    station=stations[file.stem]
    tree = etree.parse(str(file),parser=parser)
    shows=tree.xpath(".//daten/einfuegen")
    for show in shows:
        save_show(show,station)
#     empty_shows=[]
#     shows=tree.xpath(".//daten/einfuegen")
#     for show in shows:
#         production_land=show.xpath(".//stammdaten//produktion/land/text()")
#         production_land_1=show.xpath(".//stammdaten//dmbInfoset//dmbProduktionsLand/text()")
#         production_land_2=show.xpath(".//stammdaten//dmbInfoset//ProduktionsLaender/ProduktionsLand/text()")
#         if all([not production_land,not production_land_1,not production_land_2]) :
#             empty_shows.append(show)
#     for show in empty_shows:
#         save_show(show,empties,{"station":file.stem})
    print(f"{file.stem} total shows {len(shows)}")

2020_3SAT total shows 13612
2020_ALPHA total shows 16189
2020_ARD total shows 13235
2020_ARTE total shows 11431
2020_BFS total shows 13447
2020_DMAX total shows 10806
2020_DW-tv total shows 23889
2020_EinsFestival total shows 10892
2020_HF total shows 13648
2020_K1 total shows 9139
2020_K1DOKU total shows 12063
2020_KKA total shows 23703
2020_MDR total shows 15042
2020_N24DOKU total shows 10315
2020_N3_Hamburg total shows 13862
2020_N3_Radio_Bremen total shows 14726
2020_NTV total shows 22611
2020_PHOENIX total shows 11859
2020_PRO7 total shows 14579
2020_ProSiebenFun total shows 16964
2020_ProSieben_MAXX total shows 15816
2020_rbb_Berlin total shows 13550
2020_RTL total shows 10022
2020_RTL2 total shows 7180
2020_RTLplus total shows 11386
2020_RTLS total shows 14564
2020_RTL_Nitro total shows 11604
2020_SAT1 total shows 8444
2020_Sat1Gold total shows 12241
2020_sixx total shows 11814
2020_Sport1 total shows 13671
2020_SWR_BW total shows 13816
2020_SWR_SR total shows 14450
2020_tagessc

In [13]:
de_stations=db.stations_new

In [14]:
station_names=[station['name'] for station in
                        sorted(list(stations.list_collections()), key=lambda station: station['name']) if
                        station['name'] not in ['logs', 'empties']]

In [15]:
station_names

['2020_3SAT',
 '2020_ALPHA',
 '2020_ARD',
 '2020_ARTE',
 '2020_BFS',
 '2020_DMAX',
 '2020_DW-tv',
 '2020_EinsFestival',
 '2020_HF',
 '2020_K1',
 '2020_K1DOKU',
 '2020_KKA',
 '2020_MDR',
 '2020_N24DOKU',
 '2020_N3_Hamburg',
 '2020_N3_Radio_Bremen',
 '2020_NTV',
 '2020_PHOENIX',
 '2020_PRO7',
 '2020_ProSiebenFun',
 '2020_ProSieben_MAXX',
 '2020_RTL',
 '2020_RTL2',
 '2020_RTLS',
 '2020_RTL_Nitro',
 '2020_RTLplus',
 '2020_SAT1',
 '2020_SWR_BW',
 '2020_SWR_SR',
 '2020_Sat1Gold',
 '2020_Sport1',
 '2020_Tele_5',
 '2020_VOX',
 '2020_WDR',
 '2020_WELT',
 '2020_ZDF',
 '2020_ZDF_Info',
 '2020_ZDFneo',
 '2020_rbb_Berlin',
 '2020_sixx',
 '2020_tagesschau24']

In [41]:
main_station=stations['2019_3SAT']
scrapped_station=de_stations['2019_3SAT']

In [42]:
main_station.find({}).count()

<ipython-input-42-743aad3d2ec3>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  main_station.find({}).count()


0

In [43]:
scrapped_station.find({}).count()

<ipython-input-43-88099302f91b>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  scrapped_station.find({}).count()


0

In [16]:
for station_name in station_names:
    main_station=stations[station_name]
    scrapped_station=de_stations[station_name]
    print(f" total stations {main_station.find({}).count()} importable {scrapped_station.find({}).count()} in {station_name}")
    imported=0
    print('importing')
    for index, show in enumerate( scrapped_station.find({})) :
        del show['_id']
        found_show = main_station.find_one_and_update(
            {"show_id" : show['show_id']},
            {
                "$set":show
            }
        )
        if found_show:
            imported+=1
    print(f"imported {imported}")

<ipython-input-16-141b4d0c13b6>:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f" total stations {main_station.find({}).count()} importable {scrapped_station.find({}).count()} in {station_name}")


 total stations 13612 importable 6560 in 2020_3SAT
importing
imported 6560
 total stations 16189 importable 9860 in 2020_ALPHA
importing
imported 9860
 total stations 13235 importable 10965 in 2020_ARD
importing
imported 10965
 total stations 11431 importable 4960 in 2020_ARTE
importing
imported 4960
 total stations 13447 importable 12642 in 2020_BFS
importing
imported 12642
 total stations 10806 importable 3430 in 2020_DMAX
importing
imported 3430
 total stations 23889 importable 3340 in 2020_DW-tv
importing
imported 3340
 total stations 10892 importable 5811 in 2020_EinsFestival
importing
imported 5811
 total stations 13648 importable 6247 in 2020_HF
importing
imported 6247
 total stations 9139 importable 3020 in 2020_K1
importing
imported 3020
 total stations 12063 importable 1034 in 2020_K1DOKU
importing
imported 1034
 total stations 23703 importable 13589 in 2020_KKA
importing
imported 13589
 total stations 15042 importable 6102 in 2020_MDR
importing
imported 6102
 total stations 

{'_id': ObjectId('600ea27dc8e8fd3a7ca50c70'), 'show_id': ';1020463513', 'title': 'Lederstrumpf', 'subtitle': 'Der Wildtöter', 'originaltitel': '', 'sender_genre': 'Abenteuerfilm\nFernsehfilm', 'dmb_haupt_genre': 'TV-Film', 'dmb_neben_genre': 'TV-Abenteuerfilm', 'produktion_land': 'Deutschland / Frankreich', 'produktion_jahr': '1969', 'hinweissender': '', 'titel_ergaenzung': 'Der Wildtöter', 'ausstrahlung_start': '', 'ausstrahlung_sender_start': '2019-01-04T13:00:00', 'ausstrahlung_stop': '2019-01-04T14:30:00', 'ausstrahlung_dauer': '90', 'nettolaenge': '88', 'ausstrahlung_sender': '3SAT', 'produzierender_sender': '', 'dmbInfoset_ProduktionsLaender_ProduktionsLand': 'Deutschland', 'dmbFolgetitel': '', 'dmbStaffel': '', 'dmbFolge': '', 'dmbProduktionsJahr': '1969', 'dmbOriginalFolgetitel': '', 'staff': [], 'produktionsLand': [{'land': 'Deutschland', 'kfz': 'D', 'alpha3': 'DEU'}, {'land': 'Frankreich', 'kfz': 'F', 'alpha3': 'FRA'}], 'crew_united_url': '', 'crew_united_productions': [], 'c